In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import time

In [2]:
df = pd.read_csv("filmler.csv")
df

,başlık,özet,tür,vizyon_tarihi,oyuncular,yönetmen,resim_link
0,Esaretin Bedeli,"Esaretin Bedeli, Andy ve Red isimli iki mahkum...",['Dram'],11 Mart 2021,"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",['Frank Darabont'],https://tr.web.img4.acsta.net/c_310_420/pictur...
1,Hababam Sınıfı,Öğrencilik hayatları haylazlık ve tembellik üz...,['Dramatik komedi'],None,"['Münir Özkul', 'Tarık Akan', 'Adile Naşit']",['Ertem Eğilmez'],https://tr.web.img2.acsta.net/c_310_420/pictur...
2,Baba,"Baba, 40’lar ve 50’lerin Amerika’sında, bir İt...","['Polisiye', 'Dram']",None,"['Marlon Brando', 'Al Pacino', 'James Caan']",['Francis Ford Coppola'],https://tr.web.img4.acsta.net/c_310_420/medias...
3,Eşkıya,"Eşkiya, hapse düşmesine neden olan arkadaşının...",['Dram'],None,"['Şener Şen', 'Uğur Yücel', 'Yeşim Salkım']",['Yavuz Turgul'],https://tr.web.img2.acsta.net/c_310_420/pictur...
4,Cesuryürek,"Cesuryürek'te, William Wallace yaşanan büyük a...","['Biyografik', 'Dram']",None,"['Mel Gibson', 'Sophie Marceau', 'Catherine Mc...",['Mel Gibson'],https://tr.web.img3.acsta.net/c_310_420/pictur...
...,...,...,...,...,...,...,...
3412,Neşeli Pazar,Claude Massoulier’ün av yaptığı mekanda öldürü...,['Polisiye'],None,"['Fanny Ardant', 'Jean-Louis Trintignant', 'Je...",['François Truffaut'],https://tr.web.img3.acsta.net/c_310_420/pictur...
3413,Uzaydan Gelen Fırtına,"Dünya'nın önde gelen liderleri, eşi benzeri gö...","['Bilimkurgu', 'Aksiyon']",20 Ekim 2017,"['Gerard Butler', 'Jim Sturgess', 'Abbie Corni...",['Dean Devlin'],https://tr.web.img4.acsta.net/c_310_420/pictur...
3414,Carrie: Günah Tohumu,Carrie White okulda arkadaşlarının sürekli dal...,['Korku'],8 Kasım 2013,"['Chloë Grace Moretz', 'Julianne Moore', 'Judy...",['Kimberly Peirce'],https://tr.web.img4.acsta.net/c_310_420/pictur...
3415,İbo ile Güllüşah,"Sevdiği kızla, Nazlı'yla hayatını birleştirmey...",['Komedi'],None,"['Kemal Sunal', 'Gülşah Soydan', 'Ayşen Gruda']",['Atıf Yılmaz'],https://tr.web.img3.acsta.net/c_310_420/pictur...


https://www.sbert.net/docs/pretrained_models.html

# Dört farklı model

In [3]:
model1 = SentenceTransformer("distiluse-base-multilingual-cased-v1")
model2 = SentenceTransformer("distiluse-base-multilingual-cased-v2")
model3 = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
model4 = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# Embedingleri oluşturmak için özet, oyuncular, yönetmen ve türü birleştirme.

In [4]:
df = df.assign(birleşim=[str(f'''{df["özet"][i]} {str(df["oyuncular"][i]).strip("[]").replace("'", "").replace(",",".")}. {str(df["yönetmen"][i]).strip("[]").replace("'", "").replace(",",".")}. {str(df["tür"][i]).strip("[]").replace("'", "").replace(",",".")}''').lower() for i in range(len(df["başlık"]))])
df

,başlık,özet,tür,vizyon_tarihi,oyuncular,yönetmen,resim_link,birleşim
0,Esaretin Bedeli,"Esaretin Bedeli, Andy ve Red isimli iki mahkum...",['Dram'],11 Mart 2021,"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",['Frank Darabont'],https://tr.web.img4.acsta.net/c_310_420/pictur...,"esaretin bedeli, andy ve red isimli iki mahkum..."
1,Hababam Sınıfı,Öğrencilik hayatları haylazlık ve tembellik üz...,['Dramatik komedi'],None,"['Münir Özkul', 'Tarık Akan', 'Adile Naşit']",['Ertem Eğilmez'],https://tr.web.img2.acsta.net/c_310_420/pictur...,öğrencilik hayatları haylazlık ve tembellik üz...
2,Baba,"Baba, 40’lar ve 50’lerin Amerika’sında, bir İt...","['Polisiye', 'Dram']",None,"['Marlon Brando', 'Al Pacino', 'James Caan']",['Francis Ford Coppola'],https://tr.web.img4.acsta.net/c_310_420/medias...,"baba, 40’lar ve 50’lerin amerika’sında, bir i̇..."
3,Eşkıya,"Eşkiya, hapse düşmesine neden olan arkadaşının...",['Dram'],None,"['Şener Şen', 'Uğur Yücel', 'Yeşim Salkım']",['Yavuz Turgul'],https://tr.web.img2.acsta.net/c_310_420/pictur...,"eşkiya, hapse düşmesine neden olan arkadaşının..."
4,Cesuryürek,"Cesuryürek'te, William Wallace yaşanan büyük a...","['Biyografik', 'Dram']",None,"['Mel Gibson', 'Sophie Marceau', 'Catherine Mc...",['Mel Gibson'],https://tr.web.img3.acsta.net/c_310_420/pictur...,"cesuryürek'te, william wallace yaşanan büyük a..."
...,...,...,...,...,...,...,...,...
3412,Neşeli Pazar,Claude Massoulier’ün av yaptığı mekanda öldürü...,['Polisiye'],None,"['Fanny Ardant', 'Jean-Louis Trintignant', 'Je...",['François Truffaut'],https://tr.web.img3.acsta.net/c_310_420/pictur...,claude massoulier’ün av yaptığı mekanda öldürü...
3413,Uzaydan Gelen Fırtına,"Dünya'nın önde gelen liderleri, eşi benzeri gö...","['Bilimkurgu', 'Aksiyon']",20 Ekim 2017,"['Gerard Butler', 'Jim Sturgess', 'Abbie Corni...",['Dean Devlin'],https://tr.web.img4.acsta.net/c_310_420/pictur...,"dünya'nın önde gelen liderleri, eşi benzeri gö..."
3414,Carrie: Günah Tohumu,Carrie White okulda arkadaşlarının sürekli dal...,['Korku'],8 Kasım 2013,"['Chloë Grace Moretz', 'Julianne Moore', 'Judy...",['Kimberly Peirce'],https://tr.web.img4.acsta.net/c_310_420/pictur...,carrie white okulda arkadaşlarının sürekli dal...
3415,İbo ile Güllüşah,"Sevdiği kızla, Nazlı'yla hayatını birleştirmey...",['Komedi'],None,"['Kemal Sunal', 'Gülşah Soydan', 'Ayşen Gruda']",['Atıf Yılmaz'],https://tr.web.img3.acsta.net/c_310_420/pictur...,"sevdiği kızla, nazlı'yla hayatını birleştirmey..."


In [5]:
descriptions = np.array(df["birleşim"])

# Modelleri kullanarak 4 ayrı embedding oluşturma

In [6]:
bas = time.time()
description_embeddings1 = model1.encode(descriptions)
bit = time.time()
print(bit-bas)

bas = time.time()
description_embeddings2 = model2.encode(descriptions)
bit = time.time()
print(bit-bas)

bas = time.time()
description_embeddings3 = model3.encode(descriptions)
bit = time.time()
print(bit-bas)

bas = time.time()
description_embeddings4 = model4.encode(descriptions)
bit = time.time()
print(bit-bas)

198.5690176486969
187.870623588562
158.1829800605774
415.6943883895874


# Search fonksiyonu

In [7]:
movie_names = np.array(df["başlık"])

In [8]:
def search(querry, description_embeddings, model, movie_names, k):
    querry_embedding = model.encode(querry)
    results = np.array(util.dot_score(querry_embedding, description_embeddings)).reshape(-1)
    a = zip(movie_names, results)
    result_list = sorted(a, key = lambda x: x[1])[::-1]
    for i in result_list[:k]:
        print(i[0], ": ", i[1])

# Denemeler ve sonuçlar

In [9]:
querry1 = "Her biri farklı süper güçlere sahip olan aile bireylerinin normal yaşama ayak uyduramayışı ve kahramanlık yaparken yaşadıklarını anlatan animasyon filmi "

In [10]:
print("Model1:")
search(querry1, description_embeddings1, model1, movie_names, 20)
print()

print("Model2:")
search(querry1, description_embeddings2, model2, movie_names, 20)
print()

print("Model3:")
search(querry1, description_embeddings3, model3, movie_names, 20)
print()

print("Model4:")
search(querry1, description_embeddings4, model4, movie_names, 20)
print()

Model1:
Kara Kedi Ak Kedi :  0.45964986
İnanılmaz Aile 2 :  0.43913904
İntikam :  0.42294315
Şavaş Tanrısı :  0.41170657
İnanılmaz Aile :  0.41136467
Gülen Gözler :  0.41121376
Gol :  0.410612
Viva Zapata! :  0.39728242
Ocean Heaven :  0.39319947
Wallace ve Gromit Yaramaz Tavşana Karşı :  0.39191318
DC Süper Evciller Takımı :  0.39176118
Fare Şehri :  0.38651997
Bak Şu Köpeğe :  0.3859703
Kırda Bir Pazar :  0.3811906
Herkese Karşı Tek Başına :  0.37715822
Şaşkın Köpekler :  0.3770349
Yaşamak Güzel Şey :  0.37237746
Unutulmuşlar :  0.3721335
Ailem Robotlara Karşı :  0.36938888
Cennetin Çocukları :  0.36919165

Model2:
Kahramanlar Okulu :  0.36379454
İnanılmaz Aile 2 :  0.3579688
Kara Kedi Ak Kedi :  0.32677585
Şavaş Tanrısı :  0.32567313
İnanılmaz Aile :  0.32398766
Ocean Heaven :  0.31721157
Vavien :  0.31691712
DC Süper Evciller Takımı :  0.31163514
Girl Interrupted :  0.31017786
Rango :  0.3093199
Truman Show :  0.30866253
Diğer Kızkardeş :  0.3050486
Muhteşem Ambersonlar :  0.304532

In [11]:
querry2 = "Mavi yaratıkların bulunduğu gezegeni sömüren insanlar ile mavi yaratıkların savaşını anlatan, james cameronun yönettiği film"

In [12]:
print("Model1:")
search(querry2, description_embeddings1, model1, movie_names, 20)
print()

print("Model2:")
search(querry2, description_embeddings2, model2, movie_names, 20)
print()

print("Model3:")
search(querry2, description_embeddings3, model3, movie_names, 20)
print()

print("Model4:")
search(querry2, description_embeddings4, model4, movie_names, 20)
print()

Model1:
Sanctum :  0.4198127
Maymunlar Cehennemi: Başlangıç :  0.40758228
Buz Devri 5: Büyük Çarpışma :  0.4051438
Viva Zapata! :  0.38984063
Zafer Yolu :  0.38903284
Maymunlar Cehennemi: Savaş :  0.38512468
Yaratık 2 :  0.3789433
John Carter :  0.37792623
Pasifik Savaşı :  0.37684536
Titanik :  0.3744368
Ölümcül Dövüş :  0.37312335
Maymunlar Cehenneminden Kaçış :  0.3715676
Bad Taste :  0.370608
Kayıp Dünya :  0.36823803
Finch :  0.36074552
İntikam :  0.35928252
Jurassic World: Hakimiyet :  0.3559001
Yasak Bölge 9 :  0.35148644
Korkak Robert Ford'un Jesse James Suikastı :  0.34960604
Megazeka 3D :  0.34897023

Model2:
Buz Devri 5: Büyük Çarpışma :  0.34576467
Yaratık 2 :  0.30508077
Ender's Game: Uzay Oyunları :  0.30235717
Ninja Kaplumbağalar 3 :  0.3000478
Titanik :  0.28194475
Maymunlar Cehennemi: Başlangıç :  0.27936506
Jurassic World: Hakimiyet :  0.2785303
Çılgın Marslılar :  0.2749097
Warcraft: İki Dünyanın İlk Karşılaşması :  0.27458903
Transformers 3: Ay'ın Karanlık Yüzü :  0

In [13]:
querry3 = "Sihirli bir ailenin sihirli olan evinin yıkılması ve sihirsiz olduğuna inanılan aile üyesinin tüm aileyi biraraya getirmeyi başararak evi yeniden inşaları"

In [14]:
print("Model1:")
search(querry3, description_embeddings1, model1, movie_names, 20)
print()

print("Model2:")
search(querry3, description_embeddings2, model2, movie_names, 20)
print()

print("Model3:")
search(querry3, description_embeddings3, model3, movie_names, 20)
print()

print("Model4:")
search(querry3, description_embeddings4, model4, movie_names, 20)
print()

Model1:
Enkanto: Sihirli Dünya :  0.42731673
Song of the Sea :  0.38433638
Zor Hedef Fare :  0.3761852
Mary Poppins :  0.356736
Düşler Bahçesi :  0.3525993
Üç Harfliler 3: Karabüyü :  0.34346193
Around the Bend :  0.3397634
Düttürü Dünya :  0.3397193
Hadi Gidelim :  0.33547172
Harry Potter ve Zümrüdüanka Yoldaşlığı :  0.33426404
Melekler Şehri :  0.33206785
Hayalet Avcıları: Öteki Dünya :  0.33084154
Siccin 2 :  0.3299609
Siccin 4 :  0.32855755
Beterböcek :  0.3263895
Şans Kapıyı Kırınca :  0.3261573
Casper :  0.32495803
İnanılmaz Aile 2 :  0.3237867
Timecrimes :  0.32232922
Sessiz Bir Yer 2 :  0.32141092

Model2:
Enkanto: Sihirli Dünya :  0.40126127
Harry Potter ve Zümrüdüanka Yoldaşlığı :  0.37348065
Song of the Sea :  0.33709705
Siccin 2 :  0.3224438
Harry Potter ve Felsefe Taşı :  0.307539
Harry Potter ve Sırlar Odası :  0.30410862
Kötü Ruh :  0.29863614
Mary Poppins :  0.29828122
Yeti Efsanesi :  0.29462165
Fantastik Canavarlar: Grindelwald'ın Suçları :  0.29339415
Beterböcek : 

In [15]:
querry4 = "Maceralara atılmak isteyen bir balığın kaybolmasını ve bu süreçte yaşadığı maceraları anlatan animasyon filmi"

In [16]:
print("Model1:")
search(querry4, description_embeddings1, model1, movie_names, 20)
print()

print("Model2:")
search(querry4, description_embeddings2, model2, movie_names, 20)
print()

print("Model3:")
search(querry4, description_embeddings3, model3, movie_names, 20)
print()

print("Model4:")
search(querry4, description_embeddings4, model4, movie_names, 20)
print()

Model1:
Vahşi Doğa :  0.44881484
Madagaskar 2 :  0.43545246
Kayıp Balık Nemo 3D :  0.40764028
Zootropolis: Hayvanlar Şehri :  0.39255434
Arı Filmi :  0.37986955
Küçük Denizkızı Ponyo :  0.3781749
Das Boot :  0.37718576
Pasifik Savaşı :  0.37655994
Ocean Heaven :  0.36960942
Bak Şu Köpeğe :  0.36934572
M.Ö. 10.000 :  0.36233062
Robotlar :  0.3618015
Zafer Yolu :  0.35970297
Kara Kedi Ak Kedi :  0.35329944
İnanılmaz Aile :  0.35211593
Amistad :  0.35205576
Sanctum :  0.3519904
Gemide İsyan :  0.34957132
Deepwater Horizon: Büyük Felaket :  0.348589
Neşeli Ayaklar :  0.3467465

Model2:
Zootropolis: Hayvanlar Şehri :  0.2594391
Ocean Heaven :  0.25570863
Küçük Denizkızı Ponyo :  0.25526625
Kansız :  0.24598372
Denizin Ortasında :  0.23846172
Pasifik Savaşı :  0.23795313
Buz Devri 5: Büyük Çarpışma :  0.23511837
Vahşi Doğa :  0.23201427
Madagaskar 2 :  0.22993368
Geçmişi Olmayan Adam :  0.22980455
Rango :  0.22976387
Uzak :  0.22320469
Simone :  0.21796101
Jaws :  0.21710928
Looney Tunes: Ma

In [17]:
querry5 = "liseye geçen süper güçleri olan çocukları anlatan film. Babası ve annes süper güçlü olan çocuğun sonradan süper güçler kazanmasını anlatıyor."

In [18]:
print("Model1:")
search(querry5, description_embeddings1, model1, movie_names, 20)
print()

print("Model2:")
search(querry5, description_embeddings2, model2, movie_names, 20)
print()

print("Model3:")
search(querry5, description_embeddings3, model3, movie_names, 20)
print()

print("Model4:")
search(querry5, description_embeddings4, model4, movie_names, 20)
print()

Model1:
Gol :  0.4590486
Machuca :  0.44884753
İnanılmaz Aile 2 :  0.44525635
Terabithia Köprüsü :  0.44406754
Şaşkın Köpekler :  0.44163284
Şavaş Tanrısı :  0.42830247
Komando Dadı :  0.42518002
O :  0.4209288
Kara Kedi Ak Kedi :  0.41466972
Hababam Sınıfı :  0.41440275
Viva Zapata! :  0.40726137
İvan’ın Çocukluğu :  0.40415135
Penelope :  0.4028567
1900 :  0.39570922
Tepedeki Ev :  0.39566803
Çocukluk :  0.39519423
Nanny McPhee Büyük Patlama :  0.39187646
Ocean Heaven :  0.39153066
Cennetin Çocukları :  0.39087206
Ekim Düşü :  0.3899373

Model2:
Kahramanlar Okulu :  0.3568932
Shang-Chi ve On Halka Efsanesi :  0.34062207
İnanılmaz Aile 2 :  0.33492044
Akşam Yıldızı :  0.3132363
Ran :  0.30875617
Ay :  0.30866963
Sır Çocukları :  0.3078062
Ocean Heaven :  0.30379653
İpler :  0.3015908
Zoom :  0.30032444
Çocukluk :  0.30010918
Ekinoks Çiçeği :  0.29621333
Matilda :  0.29581198
Ant-Man ve Wasp :  0.2935251
O... Çocukları :  0.29065087
Şrek 2 :  0.29008883
Tepedeki Ev :  0.28984675
Şeytan

In [19]:
querry6 = "Quentin Tarantino'nun doğrusal olmayan bir zaman akışı tekniği kullanarak yönettiği, iki gangsterin maceralarını konu alan suç ve gerilim filmidir."

In [20]:
print("Model1:")
search(querry6, description_embeddings1, model1, movie_names, 40)
print()

print("Model2:")
search(querry6, description_embeddings2, model2, movie_names, 40)
print()

print("Model3:")
search(querry6, description_embeddings3, model3, movie_names, 40)
print()

print("Model4:")
search(querry6, description_embeddings4, model4, movie_names, 40)
print()

Model1:
Trendeki Kız :  0.45519504
Cinnet :  0.4161129
Dabbe 7: El-Nazar :  0.3998086
Deney :  0.39154717
Adaletin Merkezi :  0.38933292
İntikam :  0.38589448
Kanlı Toprak :  0.3847412
Korkusuz Şerifler :  0.3820477
Guguk Kuşu :  0.3790549
Bir Zamanlar Anadolu'da :  0.3770945
Zorlu İkili :  0.373395
Üçüncü Sayfa :  0.36890107
Kurt :  0.3680445
The Sting :  0.36750287
Süper Polis :  0.36616722
Karga 2 :  0.36406443
Öldüren Hatıralar :  0.360093
Angel-A :  0.3577107
Parçalanmış :  0.35719806
Viva Zapata! :  0.35477066
Zootropolis: Hayvanlar Şehri :  0.3545897
Uçurum - Asla Arkana Bakma :  0.3530922
Hoffa :  0.35305494
Brave, The :  0.35090148
Nazarín :  0.3507819
Köşe Başındakiler :  0.35047472
Kardaki İzler :  0.34871644
Testere :  0.34830937
Gizli Gerçekler :  0.3477505
Tersyüz :  0.34720647
Non-Stop :  0.34557444
Beynelmilel :  0.3454638
Panzehir :  0.34474492
Kapışma :  0.34466702
Zack Snyder's Justice League :  0.3438318
Ölüm Kitabı :  0.34315345
Escape from Alcatraz :  0.34212163
B